# Задача транспортировки офисных кресел

***Переменные***

\\( x_{ij} \\) — количество единиц товара, отправляемых из пункта \\( A \\) (Омск, Новосибирск, Томск) в пункт \\( B \\) (Нижний Новгород, Пермь, Краснодар).  

- \\( x_{11}, x_{12}, x_{13} \\) — поставки из Омска  
- \\( x_{21}, x_{22}, x_{23} \\) — поставки из Новосибирска  
- \\( x_{31}, x_{32}, x_{33} \\) — поставки из Томска  

***Целевая функция (минимизация транспортных затрат):***

\\[
Z = 4883x_{11} + 4280x_{12} + 6213x_{13} + 5327x_{21} + 4296x_{22} + 6188x_{23} + 6006x_{31} + 5030x_{32} + 7224x_{33}  -> \min
\\]

***Ограничения по производственным мощностям:***  

\\[
x_{11} + x_{12} + x_{13} = 2000 - Омск
\\]  
\\[
x_{21} + x_{22} + x_{23} = 1000 - Новосибирск
\\]  
\\[
x_{31} + x_{32} + x_{33} = 1700 - Томск
\\]  

***Ограничения по потребностям:***

\\[
x_{11} + x_{21} + x_{31} = 1400 - Нижний Новгород
\\]  
\\[
x_{12} + x_{22} + x_{32} = 1700 - Пермь
\\]  
\\[
x_{13} + x_{23} + x_{33} = 1600 - Краснодар
\\]  

***Условия неотрицательности:***  
\\[
x_{ij} >= 0
\\]  

***Стоимость перевозки за единицу товара из пунктов отправки в пункты назначения (в рублях):***  

| Откуда → Куда      | Нижний Новгород | Пермь | Краснодар |
|-------------------|----------------|-------|-----------|
| **Омск**           | 4883            | 4280  | 6213      |
| **Новосибирск**    | 5327            | 4296  | 6188      |
| **Томск**          | 6006            | 5030  | 7224      |


In [7]:
def northwest_corner_method(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    rows, cols = len(supply), len(demand)
    allocation = np.zeros((rows, cols), dtype=int)

    i, j = 0, 0
    while i < rows and j < cols:
        allocated = min(supply[i], demand[j]) #Определяем максимально возможную поставку для текущего поставщика и потребителя
        allocation[i, j] = allocated
        supply[i] -= allocated
        demand[j] -= allocated

        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1
    return allocation

Метод **северозападного угла**. Определяет первый базисный план - `allocation`

In [9]:
def calculate_cost(cost_matrix, allocation):
    total_cost = np.sum(cost_matrix * allocation)
    return total_cost

Перемножаем матрицы и возвращаем **начальную стоимость перевозки**, которую нам надо оптимизировать

In [53]:
def potentials_method(cost_matrix, allocation):
    rows, cols = cost_matrix.shape
    u = np.full(rows, None)
    v = np.full(cols, None)
    optimal_matrix = np.full((rows, cols), 0)
    u[0] = 0
    u, v, optimal_matrix = calculate_potentials(allocation, cost_matrix)# Оценка стоимости для небазисных
    delta = np.full((rows, cols), 0)
    for i in range(rows):
        for j in range(cols):
            if allocation[i, j] == 0:
                optimal_matrix[i, j] = cost_matrix[i, j] - (u[i] + v[j])
                delta[i, j] = (u[i] + v[j]) - cost_matrix[i, j]
    print()
    print(f"U{u}")
    print(f"V{v}")
    print(f"Матрица для проверки\n{optimal_matrix}")
    print()
    if np.all(optimal_matrix >= 0):
        return allocation, calculate_cost(cost_matrix, allocation)
    while np.any(optimal_matrix < 0):
            allocation = optimize_allocation(allocation, delta)
            print(allocation)
            u, v, optimal_matrix = calculate_potentials(allocation, cost_matrix)
            print()
            print(f"U{u}")
            print(f"V{v}")
            print(f"Матрица для проверки\n{optimal_matrix}")
            print()
            delta = calculate_delta(cost_matrix, u, v)
    return allocation, calculate_cost(cost_matrix, allocation)

Здесь представлен **метод потенциалов**. Мы заполняем потенциалы значениями None, а оптимальную матрицу - нулями. После чего вычисляем потенциалы и оптимальную матрицу (которую будем улучшать) с помощью метода `calculate_potentials`. Далее заполняем массив `delta` нулями. Она нам поможет понять какие клетки в оптимальной матрице стоит изменить. Вычисляем `delta` как разность потенциалов и стоимости, чтобы понять, нужно ли менять распределение. Далее проверяем все элементы оптимальной матрицы и взависимости есть ли отрицательные или нет, выполняются различные условия

In [43]:
def find_cycle(allocation, start):
    rows, cols = allocation.shape
    visited = set()
    path = []

    def dfs(node, direction):
        if node in visited:
            # Если цикл найден, возвращаем его
            if node == start:
                path.append(node)
                return True
            return False

        visited.add(node)
        path.append(node)
        i, j = node

        if direction == "row":
            for col in range(cols):
                if (allocation[i, col] > 0 and (i, col) != node) or (i, col) == start:
                    if dfs((i, col), "col"):
                        return True
        elif direction == "col":
            for row in range(rows):
                if (allocation[row, j] > 0 and (row, j) != node) or (row, j) == start:
                    if dfs((row, j), "row"):
                        return True

        # Удаляем узел, если он не часть цикла
        path.pop()
        return False

    # Запускаем поиск цикла
    found = dfs(start, "row")

    # Возвращаем цикл, если он найден
    if found and path[0] == path[-1]:
        path.pop()
        print()
        print(f"Путь{path}")
        return path
    return None

Метод `find_cycle` ищет **цикл перераспределения** в матрице `allocation` начиная с заданной точки `start`. Вложенная функция `dfs` реализует рекурсивный рекурсивный обход для поиска цикла (поиск в глубину - `DFS`). В функции проверяем в начале старт или нет, затем, если мы не в начальной точке, то проверяем сначала по строкам и столбцам наш путь, если условие выполнено метод будет вызывать сам себя. Затем удаляем узел, если он не часть цикла и возвращаем `False`. Ну и в итоге мы возвращаем либо `None`, либо `путь как список координат`

In [17]:
def optimize_allocation(allocation, delta):
    i, j = np.unravel_index(np.argmax(delta), delta.shape)
    cycle = find_cycle(allocation, (i, j))
    if not cycle:
        raise ValueError("Цикл не найден, что-то пошло не так.")

    # Чередование знаков: '+' для увеличения, '-' для уменьшения
    signs = [1 if k % 2 == 0 else -1 for k in range(len(cycle))]
    values = [allocation[x, y] for (x, y), sign in zip(cycle, signs) if sign == -1]
    theta = min(values)  # Минимум из клеток с '-' знаком
    # Обновление плана
    for (x, y), sign in zip(cycle, signs):
        allocation[x, y] += sign * theta
    return allocation

Эта функция **улучшает текущее распределение перевозок** - `allocation`. Она отвечает за корректировку перевозок таким образом, чтобы минимизировать суммарные затраты. Здесь мы находим `ячейку`, которая даёт наибольший выигрыш (`delta` — это матрица разностей между стоимостью и потенциалами). Мы должны пересчитать количество груза в ячейках цикла, следуя правилам метода потенциалов: чередуем знаки `+` и `-`. Это значит мы добавляем груз в клетки с плюсом и убираем из клеток с минусом. Далее мы выбираем все клетки цикла, из которых груз будет изыматься (sign == -1), и берём их текущие значения из `allocation`. Находим минимум со знаком минус и перераспределяем груз по циклу: увеличиваем/уменьшаем `allocation[i, j]` если `sign = +1/-1`

In [19]:
def calculate_potentials(allocation, cost_matrix):
    rows, cols = cost_matrix.shape

    u = np.full(rows, None)
    v = np.full(cols, None)
    u[0] = 0  # Начинаем с первого элемента

    # Основной цикл для вычисления потенциалов
    while None in u or None in v:
        progress = False  # Флаг для отслеживания изменений
        for i in range(rows):
            for j in range(cols):
                if allocation[i, j] > 0:  # Базисная клетка
                    if u[i] is not None and v[j] is None:
                        v[j] = cost_matrix[i, j] - u[i]
                        progress = True
                    elif v[j] is not None and u[i] is None:
                        u[i] = cost_matrix[i, j] - v[j]
                        progress = True


        # Если потенциалы не обновились, нужно использовать эвристику
        if not progress:
            min_cost, min_posit = find_min_cost_with_none(u, v, cost_matrix)
            if u[min_posit[0]] is not None and v[min_posit[1]] is None:
                v[min_posit[1]] = cost_matrix[min_posit[0], min_posit[1]] - u[min_posit[0]]
            elif u[min_posit[0]] is None and v[min_posit[1]] is not None:
                u[min_posit[0]] = cost_matrix[min_posit[0], min_posit[1]] - v[min_posit[1]]
            else: u[min_posit[0]] = 0

    # Оценка стоимости для небазисных клеток
    optimal_matrix = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            if allocation[i, j] == 0:
                optimal_matrix[i, j] = cost_matrix[i, j] - (u[i] + v[j])

    return u, v, optimal_matrix

Эта функция **вычисляет потенциалы `u` и `v`** в методе потенциалов. Потенциалы помогают определить, является ли текущее распределение грузов оптимальным. Если нет, они позволяют вычислить оценки для небазисных клеток и скорректировать план. С помощью вложенных циклов проверяем каждый элемент из `allocation`, она должна быть больше нуля. Если u[i] уже известно, а v[j] нет, то вычисляем v[j]: v[j] = $c_ij$ - u[i]. Если v[j] уже известно, а u[i] нет, то вычисляем u[i]: u[i] = $c_ij$ − v[j]. Если потенциалы не обновились используем эвристику: нужно задать начальное значение для одного из None. Вызываем функцию `find_min_cost_with_none()`, которая ищет ячейку с минимальной стоимостью перевозки и Возвращает её координаты. Потом перебираем все небазисные клетки в `allocation` и вычисляем их оценки. Возвращает потенциалы поставщиков и потребителей и оценки небазисных клеток.

In [21]:
def find_min_cost_with_none(u, v, cost_matrix):
    rows, cols = cost_matrix.shape
    min_cost = float('inf')
    min_cost_position = (-1, -1)

    # Ищем минимальную стоимость среди клеток, где u или v равны None
    for i in range(rows):
        for j in range(cols):
            if u[i] is None or v[j] is None:
                if cost_matrix[i, j] < min_cost:
                    min_cost = cost_matrix[i, j]
                    min_cost_position = (i, j)

    return min_cost, min_cost_position

Используется для **поиска ячейки с минимальной стоимостью перевозки** среди тех, где хотя бы один из потенциалов ещё не вычислен. Она применяется в методе потенциалов, когда стандартное вычисление `u` и `v` не может продолжиться. Перебираем все ячейки `cost_matrix`. Если хотя бы один из потенциалов u[i] или v[j] ещё не определён (None), то рассматриваем эту ячейку. Если оба потенциала уже известны, эту ячейку пропускаем, так как она не нужна для начальной инициализации. Если текущая стоимость перевозки `cost_matrix[i, j]` меньше текущего min_cost, обновляем: `min_cost = cost_matrix[i, j]` (новая минимальная стоимость). `min_cost_position = (i, j)` (координаты этой стоимости).

In [23]:
def calculate_delta(cost_matrix, u, v):
    rows, cols = cost_matrix.shape

    delta = np.full((rows, cols), None)
    for i in range(rows):
        for j in range(cols):
            delta[i, j] = (u[i] + v[j]) - cost_matrix[i, j]
    return delta

Эта функция вычисляет **матрицу оценок (дельта)**, которая показывает, насколько текущий план распределения отклоняется от оптимального. Если дельта:

Больше 0 → перевозка в эту ячейку невыгодна.

Равна 0 → перевозка оптимальна.

Меньше 0 → выгоднее переместить груз сюда.

In [25]:
import numpy as np
supply = [2000, 1000, 1700]
demand = [1400, 1700, 1600]
cost_matrix = np.array([
    [4883, 4280, 6213],
    [5327, 4296, 6198],
    [6006, 5030, 7224]
])

In [55]:
initial_allocation = northwest_corner_method(supply, demand)
print("Начальный опорный план:")
initial_allocation

Начальный опорный план:


array([[1400,  600,    0],
       [   0, 1000,    0],
       [   0,  100, 1600]])

In [57]:
# Шаг 2: Стоимость начального плана
initial_cost = calculate_cost(cost_matrix, initial_allocation)
print(f"Стоимость начального плана: {initial_cost}")

Стоимость начального плана: 25761600


In [59]:
# # Шаг 3: Метод потенциалов для оптимизации
optimal_allocation, optimal_cost = potentials_method(cost_matrix, initial_allocation)
print("Оптимальный план:")
print(optimal_allocation)
print(f"Минимальная стоимость перевозки: {optimal_cost}")


U[0 16 750]
V[4883 4280 6474]
Матрица для проверки
[[   0.    0. -261.]
 [ 428.    0. -292.]
 [ 373.    0.    0.]]

Путь[(1, 2), (1, 1), (2, 1), (2, 2)]
[[1400  600    0]
 [   0    0 1000]
 [   0 1100  600]]

U[0 -276 750]
V[4883 4280 6474]
Матрица для проверки
[[   0.    0. -261.]
 [ 720.  292.    0.]
 [ 373.    0.    0.]]

Путь[(0, 2), (0, 1), (2, 1), (2, 2)]
[[1400    0  600]
 [   0    0 1000]
 [   0 1700    0]]

U[0 -15 750]
V[4883 4280 6213]
Матрица для проверки
[[  0.   0.   0.]
 [459.  31.   0.]
 [373.   0. 261.]]

Оптимальный план:
[[1400    0  600]
 [   0    0 1000]
 [   0 1700    0]]
Минимальная стоимость перевозки: 25313000
